# Combined leaf and wood Xylariaceae analysis

We will use our combined biom table of leaf and wood endophytes to ask some questions about the roles of both leaf- and sapwood-inhabiting endophytes on nearby stromata, focusing on the family Xylariaceae.

We begin by [stabilizing the variance](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003531of) of our combined-leaf-and-wood-endophyte table, using the [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) package in R. 

In [6]:
library('phyloseq')
library('DESeq2')

In [2]:
## import table:
biom95 <- import_biom('combo_95_wMeta.biom')
## fix taxonomy headers:
colnames(tax_table(biom95)) <- c("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")

Our biom table:

In [5]:
print(biom95)

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 10269 taxa and 232 samples ]
sample_data() Sample Data:       [ 232 samples by 2 sample variables ]
tax_table()   Taxonomy Table:    [ 10269 taxa by 7 taxonomic ranks ]


ERROR: Error in if (getOption("jupyter.rich_display")) {: argument is of length zero
